<a href="https://colab.research.google.com/github/jacobrast/11785-hw4p1/blob/main/HW4P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [14]:
def log_softmax(x, axis):
    ret = x - np.max(x, axis=axis, keepdims=True)
    lsm = np.log(np.sum(np.exp(ret), axis=axis, keepdims=True))
    return ret - lsm


def array_to_str(arr, vocab):
    return " ".join(vocab[a] for a in arr)


def test_prediction(out, targ):
    out = log_softmax(out, 1)
    nlls = out[np.arange(out.shape[0]), targ]
    nll = -np.mean(nlls)
    return nll

def test_generation(inp, pred, vocab):
    outputs = u""
    for i in range(inp.shape[0]):
        w1 = array_to_str(inp[i], vocab)
        w2 = array_to_str(pred[i], vocab)
        outputs += u"Input | Output #{}: {} | {}\n".format(i, w1, w2)
    return outputs

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!tar -xf "/content/drive/MyDrive/Colab Notebooks/11785/HW4/handout_hw4.tar"

In [17]:
# load all that we need

dataset = np.load('handout/dataset/wiki.train.npy', allow_pickle=True)
devset = np.load('handout/dataset/wiki.valid.npy', allow_pickle=True)
fixtures_pred = np.load('handout/fixtures/prediction.npz')  # dev
fixtures_gen = np.load('handout/fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('handout/fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('handout/fixtures/generation_test.npy')  # test
vocab = np.load('handout/dataset/vocab.npy')

In [18]:
#vocab = np.array([1,2,3,4,5])
#dataset = [np.array([1,2,3,4,5])]

In [19]:
# data loader

class DataLoaderForLanguageModeling(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, seq_length, shuffle=True):
        if shuffle:
            np.random.shuffle(dataset)
        self.data = np.concatenate(dataset)
        self.batch_size = batch_size
        self.seq_length = seq_length


    def __len__(self):
        return int((self.data.shape[0] - self.seq_length - 1) / (self.batch_size * self.seq_length))
  

    def __iter__(self):
        """
            You may implement some of the techniques in https://arxiv.org/pdf/1708.02182.pdf
            example: Variable length backpropagation sequences (Section 4.1)
        """
        # 1. Randomly shuffle all the articles from the WikiText-2 dataset.
        # 2. Concatenate all text in one long string.
        # 3. Group the sequences into batches.
        # 4. Run a loop that returns a tuple of (input, label) on every iteration with yield.

        for i in range(0, self.data.shape[0] - self.seq_length - 1, self.batch_size * self.seq_length):
            #Question: what data format should I use to return the batch? A: list
            #Question: how do I deal with the last item? A: if statement
            inputs = []
            targets = []

            if (i + self.batch_size * self.seq_length > self.data.shape[0]):
                #batch_size = data.shape[0] - i
                #throw data away
                break

            ind = i

            for j in range(self.batch_size):
                  inputs.append(self.data[ind:ind+self.seq_length])
                  targets.append(self.data[ind+1:ind+1+self.seq_length])
                  ind = ind + self.seq_length            
            
            inputs = torch.from_numpy(np.stack(inputs))
            targets = torch.from_numpy(np.stack(targets))
            yield inputs, targets      

In [20]:
#Code to test my dataloader implementation

loader = DataLoaderForLanguageModeling(
    dataset=dataset, 
    batch_size=1,
    seq_length=3,
    shuffle=True
)

j = 0
i, (inputs, targets)  = next(enumerate(loader))

print(inputs)
print(targets)

tensor([[ 1420,   817, 12624]], dtype=torch.int32)
tensor([[  817, 12624,  6232]], dtype=torch.int32)


In [21]:
# model

# Let the embedding size be varaible
class Encoder(nn.Module):
    #def __init__(self, emb_size, hidden_size, vocab_size, pad_idx, num_layers, bidirectional):
    def __init__(self, emb_size, hidden_size, vocab_size, num_layers, bidirectional):
        super(Encoder, self).__init__()
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size

        self.embeddings = nn.Embedding(self.vocab_size, self.emb_size, padding_idx=0)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)

    def forward(self, x):
        embeds = self.embeddings(x)
        h, _ = self.lstm(embeds)

        return h


class Model(nn.Module):
    """
        The model should have the following layers:
          1) Embedding layer. We can use nn.Embedding.
          2) Encoder. This will be a multilayer LSTM (to begin)
          3) Decoder. Another multilayer LSTM.

        Questions:
          1. What should I use as the dimensions of my word embedding?
            a: Let this be variable. We can set it as a parameter.
          2. How to define my encoder and decoder?
            a: For now, just use a standard MLP for the decoder.
          3. Can I train word embeddings in my dataset?
            a: Not sure what this means. We can train word embeddings in the encoder, yes. For the encoder/decoder model, word embeddings
               may or may not be shared between encoder and decoder. Probably should be, but might work without?
          4. Where is the sequence length defined?
    """
    def __init__(self, vocab_size:int, embedding_dim:int, hidden_size:int):
        super(Model, self).__init__()
        encoder_layers = 2
        bidirectional = True
        self.encoder = Encoder(embedding_dim, hidden_size, vocab_size, encoder_layers, bidirectional).to(device)

        if bidirectional:
            # nn.linear input shape: B * input_size
            # nn.linear output shape: B * output_size
            #self.mlp = nn.linear(2 * encoder_layers * hidden_size, vocab_size)
            self.mlp = nn.Linear(2 * hidden_size, vocab_size).to(device)

        #else:
        # throw error for now

        self.model = nn.Sequential(self.encoder, self.mlp).to(device)


    def forward(self, x):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)

        # What are the dimensions of x?
        # How do I run only the last hidden state through the MLP?

        # h_n has dimensions D * num_layers, N, H_out
        #   where num_layers is set to 2 (for now), and D is 2 for bidirectional
        #   N is batch size
        #   h_out is the hidden size


        # #Need to modify. This will only predict the final word. We need to predict the next word for each word in the sequence.
        # _, (hidden, cell) = self.encoder(x)
        # out = self.MLP(hidden)  

        return self.model(x)

In [22]:
# # Code to test model
# model = Model(vocab.size, embedding_dim=100, hidden_size=100).to(device)

# loader = DataLoaderForLanguageModeling(
#     dataset=dataset, 
#     batch_size=2,
#     seq_length=3,
#     shuffle=True
# )
# criterion = nn.CrossEntropyLoss()
# data = enumerate(loader)

In [23]:
# batch_num, (inputs, targets) = next(data)

In [24]:
# inputs = inputs.to(device)
# outputs = model(inputs)
# targets = targets.to(device)

# print(outputs)

# targets = torch.flatten(targets).to(torch.int64)
# outputs = torch.flatten(outputs, start_dim=0, end_dim=1)

# loss = criterion(outputs, targets)

# print(loss)

# #loss.backward()

In [32]:
# model trainer

class Trainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # TODO: Define your optimizer and criterion here
        # feel free to define a learning rate scheduler as well if you want
        self.optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        batch_bar = tqdm(total=len(self.loader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5)
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
            batch_bar.set_postfix(
                loss="{:.04f}".format(float(epoch_loss / (batch_num + 1)))
                )
            batch_bar.update()
        epoch_loss = epoch_loss / (batch_num + 1)
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
            
            :return 
                    (float) loss value
        """

        self.optimizer.zero_grad() # Zero gradients

        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = self.model(inputs)

        targets = torch.flatten(targets).to(torch.int64)
        outputs = torch.flatten(outputs, start_dim=0, end_dim=1)

        loss = self.criterion(outputs, targets)

        # Update no. of correct predictions & loss as we iterate
        #num_correct += int((torch.argmax(outputs, axis=1) == targets).sum())
        #total_loss += float(loss.item())
        
        loss.backward()
        self.optimizer.step()

        return loss

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.predict(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        #generated_logits = TestLanguageModel.generate(fixtures_gen, 10, self.model) # generated predictions for 10 words
        #generated_logits_test = TestLanguageModel.generate(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        #generated = test_generation(fixtures_gen, generated_logits, vocab)
        #generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        #self.generated.append(generated)
        #self.generated_test.append(generated_test)
        #self.generated_logits.append(generated_logits)
        #self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.predict(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        self.epochs += 1

        nll = 0
        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        #np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        #np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        #with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
        #    fw.write(self.generated[-1])
        #with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
        #    fw.write(self.generated_test[-1])


In [33]:
from re import X
class TestLanguageModel:
    def predict(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits

            Question: What are logits?
        """

        model.eval()

        x = torch.from_numpy(inp)
        x = x.to(device)

        y = model(x)

        out = y[:,-1,:]

        return out.detach().cpu().numpy()

        
    def generate(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """        
        raise NotImplemented
        

In [34]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 200
BATCH_SIZE = 32
EMB_DIM = 32
HIDDEN_SIZE = 32

In [35]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1669375228


In [36]:
model = Model(vocab.size, embedding_dim=EMB_DIM, hidden_size=HIDDEN_SIZE)

loader = DataLoaderForLanguageModeling(
    dataset=dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    seq_length=3
)
trainer = Trainer(
    model=model, 
    loader=loader, 
    max_epochs=NUM_EPOCHS, 
    run_id=run_id
)

In [37]:
torch.cuda.empty_cache()

In [ ]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

[TRAIN]  Epoch [1/200]   Loss: 6.2768
[VAL]  Epoch [1/200]   Loss: 6.2447


[TRAIN]  Epoch [2/200]   Loss: 5.2673
[VAL]  Epoch [2/200]   Loss: 6.1308


[TRAIN]  Epoch [3/200]   Loss: 4.9561
[VAL]  Epoch [3/200]   Loss: 6.0339


[TRAIN]  Epoch [4/200]   Loss: 4.7293
[VAL]  Epoch [4/200]   Loss: 5.9852


[TRAIN]  Epoch [5/200]   Loss: 4.5391
[VAL]  Epoch [5/200]   Loss: 5.9680


...

[TRAIN]  Epoch [8/200]   Loss: 4.0799
[VAL]  Epoch [8/200]   Loss: 5.9785


[TRAIN]  Epoch [9/200]   Loss: 3.9528
[VAL]  Epoch [9/200]   Loss: 5.9736


[TRAIN]  Epoch [10/200]   Loss: 3.8370
[VAL]  Epoch [10/200]   Loss: 5.9544


[TRAIN]  Epoch [11/200]   Loss: 3.7306
[VAL]  Epoch [11/200]   Loss: 5.9422


Train:  44%|████▎     | 9454/21621 [01:02<01:22, 148.14it/s, loss=3.6579]

In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output